In [15]:
import numpy as np
import yfinance
from sklearn.preprocessing import MinMaxScaler
stocks=yfinance.download("AAPL", start="2015-01-01", end="2025-01-01")

[*********************100%***********************]  1 of 1 completed


In [18]:
stock_closed=np.array(stocks["Close"])
mms = MinMaxScaler((0, 1))
new_stock_closed=mms.fit_transform(stock_closed)

array([[0.01531497],
       [0.01243694],
       [0.0124463 ],
       ...,
       [0.98560779],
       [0.97138334],
       [0.9639145 ]])

In [26]:
price_days=60
features=[]
target=[]
for i in range(len(new_stock_closed)-price_days):
    features.append(new_stock_closed[i:i+price_days])
    target.append(new_stock_closed[i+price_days])

features=np.array(features)
target=np.array(target)

[[[0.01531497]
  [0.01243694]
  [0.0124463 ]
  ...
  [0.0297054 ]
  [0.02877666]
  [0.03170359]]

 [[0.01243694]
  [0.0124463 ]
  [0.01383858]
  ...
  [0.02877666]
  [0.03170359]
  [0.02988364]]

 [[0.0124463 ]
  [0.01383858]
  [0.0177071 ]
  ...
  [0.03170359]
  [0.02988364]
  [0.02971477]]

 ...

 [[0.86367557]
  [0.85902308]
  [0.86375932]
  ...
  [0.98426514]
  [0.99655943]
  [1.        ]]

 [[0.85902308]
  [0.86375932]
  [0.84234143]
  ...
  [0.99655943]
  [1.        ]
  [0.98560779]]

 [[0.86375932]
  [0.84234143]
  [0.85944224]
  ...
  [1.        ]
  [0.98560779]
  [0.97138334]]]


In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(price_days, 1)),
    Dropout(0.2),
    LSTM(units=50, return_sequences=True),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer="adam", loss="mean_squared_error")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [35]:
from sklearn.model_selection import train_test_split as tts
train_features, test_features, train_target, test_target = tts(features, target, train_size=0.8, random_state=42)

model.fit(train_features, train_target, epochs=50, batch_size=32)

Epoch 1/50


ValueError: Attr 'Toutput_types' of 'OptionalFromValue' Op passed list of length 0 less than minimum 1.